# Definitive, Production-Grade Auto-Training Pipeline (Stable Grounded-SAM)

This notebook implements the complete AutoDistill workflow using the stable and reliable **Grounded-SAM** base model. It is architected with a high-precision, **Iterative, Per-Class Detection** strategy to ensure the highest possible accuracy for the auto-labeling phase.

### 1. Setup Environment (Robust & Stable Installation)

In [ ]:
print("🚨 Starting Stable AutoDistill Workflow Installation...")

from google.colab import drive
drive.mount('/content/drive')

import os
HOME = os.getcwd()
print(f"Working Directory: {HOME}")

print("📦 Installing All Required Packages...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q opencv-python-headless matplotlib numpy timm transformers huggingface-hub
!pip install -q segment-anything
# Install the STABLE versions of the autodistill ecosystem
!pip install -q autodistill autodistill-grounded-sam autodistill-yolov8 roboflow supervision

print("🎯 Downloading SAM Model Checkpoint...")
SAM_CHECKPOINT_PATH = "sam_vit_h_4b8939.pth"
if not os.path.exists(SAM_CHECKPOINT_PATH):
    !wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

import torch
torch.cuda.empty_cache()
print("\n🎉 INSTALLATION COMPLETE!")

### 2. Configuration

In [ ]:
import os
# --- Main Paths ---
BASE_DRIVE_PATH = "/content/drive/MyDrive/"
TEMPLATE_FOLDER_PATH = os.path.join(BASE_DRIVE_PATH, "hvac_templates/")
UNLABELED_IMAGES_PATH = os.path.join(BASE_DRIVE_PATH, "hvac_example_images/")
DATASET_OUTPUT_PATH = os.path.join(HOME, "hvac_autodistill_dataset/")
TRAINING_OUTPUT_PATH = os.path.join(BASE_DRIVE_PATH, "hvac_yolov8_training/")

os.makedirs(DATASET_OUTPUT_PATH, exist_ok=True)
os.makedirs(TRAINING_OUTPUT_PATH, exist_ok=True)

# --- Auto-Labeling Parameters ---
# Use high confidence with the per-class detection method
BOX_THRESHOLD = 0.4
TEXT_THRESHOLD = 0.3

### 3. Phase 1: High-Precision Auto-Labeling

In [ ]:
from autodistill.detection import CaptionOntology
from autodistill_grounded_sam import GroundedSAM
from autodistill.core.dataset import DetectionDataset
import os
import glob
import supervision as sv
import cv2
import numpy as np
from collections import Counter

print("Building HVAC Ontology from template library...")

all_template_files = glob.glob(os.path.join(TEMPLATE_FOLDER_PATH, '*.png')) + glob.glob(os.path.join(TEMPLATE_FOLDER_PATH, '*.PNG'))
if not all_template_files: raise FileNotFoundError(f"FATAL: No template files found in {TEMPLATE_FOLDER_PATH}.")

ontology_mapping = {}
for f_path in all_template_files:
    clean_name = os.path.splitext(os.path.basename(f_path))[0].replace('template_', '').replace('_', ' ')
    ontology_mapping[clean_name] = clean_name

ontology = CaptionOntology(ontology_mapping)
print(f"\n✅ Ontology Created with {len(ontology.classes())} classes.")

print("\nInitializing Grounded-SAM 'Teacher' model...")
base_model = GroundedSAM(ontology=ontology, box_threshold=BOX_THRESHOLD, text_threshold=TEXT_THRESHOLD)

print(f"\nStarting auto-labeling on images in: {UNLABELED_IMAGES_PATH}")

image_paths = glob.glob(os.path.join(UNLABELED_IMAGES_PATH, '*'))
dataset = DetectionDataset(classes=ontology.classes(), base_dir=DATASET_OUTPUT_PATH)
total_detections_by_class = Counter()
images_processed = 0

for img_path in image_paths:
    print(f"\n--- Processing: {os.path.basename(img_path)} ---")
    try:
        image = cv2.imread(img_path)
        if image is None: continue
        
        # The key to precision: run predict() for EACH class
        all_detections_for_image = []
        for i, class_name in enumerate(ontology.classes()):
            detections = base_model.predict(image, prompt=class_name)
            detections.class_id = np.full(len(detections), i)
            all_detections_for_image.append(detections)
        
        if all_detections_for_image:
            final_detections = sv.Detections.merge(all_detections_for_image)
            dataset.add_detection(image_path=img_path, detections=final_detections)
            images_processed += 1
            
            class_counts = Counter([ontology.classes()[cid] for cid in final_detections.class_id])
            total_detections_by_class.update(class_counts)
            
            avg_confidence = np.mean(final_detections.confidence) if len(final_detections.confidence) > 0 else 0
            print(f"  -> ✅ SUCCESS: Found {len(final_detections)} symbols.")
            print(f"     - Average Confidence: {avg_confidence:.2f}")
        else:
            print("  -> INFO: No symbols detected in this image.")
    except Exception as e:
        print(f"  -> ❌ ERROR: Failed to process image. Error: {e}")

print("\n" + "="*60)
print("--- PHASE 1 COMPLETE: Auto-Labeling Finished ---")
print(f"Dataset saved at: {DATASET_OUTPUT_PATH}")
print("="*60)

### 4. Phase 2: Manual Approval Gate

In [ ]:
import supervision as sv
import os
import matplotlib.pyplot as plt
import cv2

print("🔍 Starting review of the auto-labeled dataset...")

IMAGES_DIR = os.path.join(DATASET_OUTPUT_PATH, "train/images")
LABELS_DIR = os.path.join(DATASET_OUTPUT_PATH, "train/labels")
DATA_YAML_PATH = os.path.join(DATASET_OUTPUT_PATH, "data.yaml")

try:
    review_dataset = sv.DetectionDataset.from_yolo(
        images_directory_path=IMAGES_DIR,
        annotations_directory_path=LABELS_DIR,
        data_yaml_path=DATA_YAML_PATH
    )
    print(f"  -> Successfully loaded dataset with {len(review_dataset)} images.")
except Exception as e:
    raise FileNotFoundError(f"FATAL: Could not load the dataset. Error: {e}.")

SAMPLE_SIZE = 5
if len(review_dataset) < SAMPLE_SIZE: SAMPLE_SIZE = len(review_dataset)

if SAMPLE_SIZE > 0:
    box_annotator = sv.BoxAnnotator(thickness=2)
    label_annotator = sv.LabelAnnotator(text_thickness=1, text_scale=0.5, text_padding=3)
    dataset_items = list(review_dataset)
    sample_items = dataset_items[:SAMPLE_SIZE]
    annotated_images, titles = [], []
    
    for image_path, detections in sample_items:
        image = cv2.imread(image_path)
        labels = [f"{review_dataset.classes[class_id]}" for class_id in detections.class_id]
        annotated_image = box_annotator.annotate(scene=image.copy(), detections=detections)
        annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)
        annotated_images.append(annotated_image)
        titles.append(os.path.basename(image_path))

    print(f"  -> Displaying a sample of {SAMPLE_SIZE} auto-labeled images for review...")
    sv.plot_images_grid(images=annotated_images, titles=titles, grid_size=(1, SAMPLE_SIZE), size=(20, 10))
else:
    print("  -> No images were labeled. Nothing to review.")

print("\n--- Review Complete ---")
user_approval = input("\n🛑 Please review the auto-labeled images above. Do you approve this dataset for training? (yes/no): ")

PROCEED_TO_TRAINING = (user_approval.lower() == 'yes')

if PROCEED_TO_TRAINING:
    print("\n✅ Dataset approved. Proceeding to Phase 3: Training.")
else:
    print("\n❌ Dataset rejected. Training will be skipped. ---")

### 5. Phase 3: Train the YOLOv8 "Student" Model

In [ ]:
if PROCEED_TO_TRAINING:
    from autodistill_yolov8 import YOLOv8
    import os
    import torch
    from ultralytics.nn.modules import C2f, Detect, Bottleneck, Conv, ConvTranspose, DFL
    import locale

    locale.getpreferredencoding = lambda: "UTF-8"

    TRAIN_DATASET_PATH = os.path.join(DATASET_OUTPUT_PATH, "data.yaml")
    EPOCHS = 50

    print("\nInitializing YOLOv8 'Student' model with security context...")

    SAFE_GLOBALS = [C2f, Detect, Bottleneck, Conv, ConvTranspose, DFL, torch.nn.ModuleList]

    try:
        with torch.serialization.safe_globals(SAFE_GLOBALS):
            target_model = YOLOv8("yolov8n.pt")
        print("YOLOv8 model initialized successfully.")

        print(f"\nStarting training for {EPOCHS} epochs...")
        target_model.train(
            data_path=TRAIN_DATASET_PATH, 
            epochs=EPOCHS,
            project=TRAINING_OUTPUT_PATH
        )
        print("\n--- PHASE 3 COMPLETE: YOLOv8 training finished. ---")
    except Exception as e:
        print(f"❌ YOLOv8 Training Failed. Error: {e}")
        PROCEED_TO_TRAINING = False
else:
    print("Training skipped as per user input.")

### 6. Phase 4: Inference with Your Custom-Trained Expert Model

In [ ]:
if PROCEED_TO_TRAINING:
    from ultralytics import YOLO
    import glob
    import os
    import cv2
    from IPython.display import Image, display

    run_folders = sorted(glob.glob(os.path.join(TRAINING_OUTPUT_PATH, 'train*')))
    if not run_folders:
        raise FileNotFoundError(f"FATAL: No training runs found.")
    latest_run_folder = run_folders[-1]
    TRAINED_MODEL_PATH = os.path.join(latest_run_folder, 'weights/best.pt')

    print(f"\nLoading your custom-trained HVAC expert model from: {TRAINED_MODEL_PATH}")
    model = YOLO(TRAINED_MODEL_PATH)

    image_paths = glob.glob(os.path.join(UNLABELED_IMAGES_PATH, '*'))
    inference_image_path = image_paths[-1]
    print(f"Running inference on: {os.path.basename(inference_image_path)}")

    results = model(inference_image_path)
    annotated_frame = results[0].plot()

    INFERENCE_OUTPUT_FOLDER = os.path.join(BASE_DRIVE_PATH, "hvac_inference_results/")
    os.makedirs(INFERENCE_OUTPUT_FOLDER, exist_ok=True)

    output_filename = f"inference_result_{os.path.basename(inference_image_path)}.png"
    output_path = os.path.join(INFERENCE_OUTPUT_FOLDER, output_filename)
    cv2.imwrite(output_path, annotated_frame)

    print(f"\nInference complete. Found {len(results[0].boxes)} symbols.")
    print(f"Saved inference result to: {output_path}")
    display(Image(filename=output_path, width=800))
    print("\n--- PIPELINE COMPLETE ---")
else:
    print("Inference skipped as per user input.")